In [1]:
import numpy as np
from skimage.io import imshow, imread
from skimage.color import rgb2gray, gray2rgb
import matplotlib.pyplot as plt
import imutils
from skimage.feature import match_template
from skimage.feature import peak_local_max
import cv2
import MTM
from MTM import matchTemplates, drawBoxesOnRGB
from scipy import signal
import os

# Sklearn based fast, normalized cross-correlation

In [ ]:
sample = imread('test_image_2.png')
sample_g = rgb2gray(sample)
fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(sample)
ax[1].imshow(sample_g,cmap='gray')
ax[0].set_title('Colored Image',fontsize=15)
ax[1].set_title('Grayscale Image',fontsize=15)
plt.show()
for i in os.listdir("ExpertTemplate"):
    template = imread(os.path.join(os.getcwd(),"ExpertTemplate",i))
    template_g = rgb2gray(template)
    sample_mt = match_template(sample_g, template_g)
    fig, ax = plt.subplots(1,2,figsize=(10,10))
    ax[0].imshow(sample_g,cmap='gray')
    ax[1].imshow(sample_mt,cmap='gray')
    ax[0].set_title('Grayscale',fontsize=15)
    ax[1].set_title('Template Matching',fontsize=15);
    fig, ax = plt.subplots(1,2,figsize=(10,10))
    ax[0].imshow(sample)
    ax[1].imshow(sample)
    patch_width, patch_height = template_g.shape
    for x, y in peak_local_max(sample_mt, threshold_abs=0.6):
        rect = plt.Rectangle((y, x), patch_height, patch_width, color='r', 
                             fc='none')
        ax[1].add_patch(rect)
    ax[0].set_title('Colored Image',fontsize=15)
    ax[1].set_title('Template Matched',fontsize=15)
    plt.show()

# ORB based feature matching

In [ ]:
img1 = cv2.imread('test_image_2.png',cv2.IMREAD_GRAYSCALE)
for i in os.listdir("ExpertTemplate"):
    img2 = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread('template_image.png',cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    #cv.NORM_HAMMING method
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)
    img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(15, 12), dpi=80)
    plt.imshow(img3),plt.show()
    # cv.NORM_L2 method
    bf2 = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf2.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)
    img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(15, 12), dpi=80)
    plt.imshow(img3),plt.show()

# SIFT with BFMatcher based fature matching

In [ ]:
img1 = cv2.imread('test_image_2.png',cv2.IMREAD_GRAYSCALE)
for i in os.listdir("ExpertTemplate"):
    img2 = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(15, 12), dpi=80)
    plt.imshow(img3),plt.show()

# SIFT with FLANN based fature matching

In [ ]:
img1 = cv2.imread('test_image_2.png',cv2.IMREAD_GRAYSCALE)
for i in os.listdir("ExpertTemplate"):
    img2 = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50) 
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i]=[1,0]
    draw_params = dict(matchColor = (0,255,0),
                       singlePointColor = (255,0,0),
                       matchesMask = matchesMask,
                       flags = cv2.DrawMatchesFlags_DEFAULT)
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    plt.figure(figsize=(15, 12), dpi=80)
    plt.imshow(img3,),plt.show()

# Multi scale template matching

In [ ]:
for i in os.listdir("ExpertTemplate"):
    template = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
    template = cv2.Canny(template, 50, 200)
    (tH, tW) = template.shape[:2]
    plt.imshow(template)
    plt.show()
    image = cv2.imread('test_image_2.png') 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    found = None
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
        resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
        r = gray.shape[1] / float(resized.shape[1])
        if resized.shape[0] < tH or resized.shape[1] < tW:
            break
        edged = cv2.Canny(resized, 50, 200)
        result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
        (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
        clone = np.dstack([edged, edged, edged])
        cv2.rectangle(clone, (maxLoc[0], maxLoc[1]),(maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
        plt.imshow(clone)
        plt.show()
        if found is None or maxVal > found[0]:
            found = (maxVal, maxLoc, r)
    (_, maxLoc, r) = found
    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
    plt.imshow(image)
    plt.show()

# Template matching with different opencv methods

In [ ]:
img = cv2.imread('test_image_2.png',0)
img2 = img.copy()
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
for i in os.listdir("ExpertTemplate"):
    template = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
    w, h = template.shape[::-1]
    for meth in methods:
        img = img2.copy()
        method = eval(meth)
        res = cv2.matchTemplate(img,template,method)
        fig, ax = plt.subplots(1,2,figsize=(20,20))
        ax[0].imshow(img)
        ax[1].imshow(img)
        patch_width, patch_height = img2.shape
        for x, y in peak_local_max(res, threshold_abs=0.6):
            rect = plt.Rectangle((y, x), patch_height, patch_width, color='r', fc='none')
            ax[1].add_patch(rect)
        ax[0].set_title('Grayscale',fontsize=15)
        ax[1].set_title('Template Matched',fontsize=15);

/conda/anaconda3/envs/DataCreaterV1/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


# Template matching using MTM package

In [ ]:
img = cv2.imread('test_image_2.png',0)
for i in os.listdir("ExpertTemplate"):
    template = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)

    listTemplate = [('parle', template)]

    threshold = 0.1

    Hits = matchTemplates(listTemplate, img, score_threshold=threshold, method=cv2.TM_CCOEFF, maxOverlap=0)

    Overlay = drawBoxesOnRGB(img, Hits, showLabel=True)
    plt.imshow(Overlay)
    plt.show()


    Hits2 = matchTemplates(listTemplate, img, score_threshold=threshold, method=cv2.TM_CCOEFF_NORMED, maxOverlap=0)

    Overlay2 = drawBoxesOnRGB(img, Hits2, showLabel=True)
    plt.imshow(Overlay2)
    plt.show()


    Hits3 = matchTemplates(listTemplate, img, score_threshold=threshold, method=cv2.TM_CCORR, maxOverlap=0)

    Overlay3 = drawBoxesOnRGB(img, Hits3, showLabel=True)
    plt.imshow(Overlay3)
    plt.show()


    Hits4 = matchTemplates(listTemplate, img, score_threshold=threshold, method=cv2.TM_CCORR_NORMED, maxOverlap=0)

    Overlay4 = drawBoxesOnRGB(img, Hits4, showLabel=True)
    plt.imshow(Overlay4)
    plt.show()


    Hits6 = matchTemplates(listTemplate, img, score_threshold=threshold, method=cv2.TM_SQDIFF_NORMED, maxOverlap=0)

    Overlay6 = drawBoxesOnRGB(img, Hits6, showLabel=True)
    plt.imshow(Overlay6)
    plt.show()

# Scipy based correlation between image and template

In [ ]:
img = cv2.imread('test_image_2.png',0)
for i in os.listdir("ExpertTemplate"):
    template = cv2.imread(os.path.join(os.getcwd(),"ExpertTemplate",i),cv2.IMREAD_GRAYSCALE)
    corr = signal.correlate2d(img, template, boundary='symm', mode='same')
    y, x = np.unravel_index(np.argmax(corr), corr.shape)
    fig, (ax_orig, ax_template, ax_corr) = plt.subplots(3, 1,figsize=(6, 15))
    ax_orig.imshow(img, cmap='gray')
    ax_orig.set_title('Original')
    ax_orig.set_axis_off()
    ax_template.imshow(template, cmap='gray')
    ax_template.set_title('Template')
    ax_template.set_axis_off()
    ax_corr.imshow(corr, cmap='gray')
    ax_corr.set_title('Cross-correlation')
    ax_corr.set_axis_off()
    ax_orig.plot(x, y, 'ro')
    fig.show()